##### Celery
- python에서 사용하는 비동기 작업큐
- message broker 가 python process를 연결해서 process를 컨트롤해 줌
- message broker(asynchronous queue) - redis
- install package
    - mac
        - `$ pip3 install celery redis`
    - windows
        - 관리자 권한으로 cmd 창에서 실행
        - celery 4.x 버전은 windows를 지원하지 않습니다.
        - `> conda install -c conda-forge celery==3.1.25`
        - `> conda install -c conda-forge redis-py`
- reference : http://www.celeryproject.org/

##### Redis
- mac
    - `$ brew install redis`
    - `$ redis-server`
- window
    - https://github.com/MSOpenTech/redis/releases download
        - radis-x64-3.x.msi
        - 설치후 재부팅
    - `C:\ProgramFiles\Redis> redis-cli.exe`
    - `127.0.0.1:6379> shutdown`
    - `not connected> exit`
    - `C:\ProgramFiles\Redis> redis-server.exe`

In [1]:
import redis

r = redis.StrictRedis(host='localhost', port=6379, db=0)

# set key and value
result = r.set('foo', 'bar')
print("set", result)

set True


In [2]:
# get value 
result = r.get('foo')
print("get", result)

get b'bar'


In [3]:
# decode ascii
result = result.decode('ascii')
print("get(after decode)", result)

get(after decode) bar


##### start celery
- make task.py
- start celery
    - mac
        - `$ celery -A task worker`
    - windows
        - `$ celery -A task worker -l info`
- run celery

- make task.py

In [14]:
# prime_number
def prime_number(n):
    prime_list = []
    for num1 in range(1, n+1):
        is_prime = True
        for num2 in range(2, num1):
            if num1 % num2 == 0:
                is_prime = False
        if is_prime:
            prime_list.append(num1)
    return prime_list

In [15]:
%time len(prime_number(10000))

CPU times: user 3.21 s, sys: 4.87 ms, total: 3.21 s
Wall time: 3.21 s


1230

In [16]:
%time len(prime_number(20000))

CPU times: user 12.7 s, sys: 14.3 ms, total: 12.7 s
Wall time: 12.7 s


2263

In [17]:
%%writefile task.py

from celery import Celery

BROKER_URL = 'redis://localhost:6379/0'
CELERY_RESULT_BACKEND = 'redis://localhost:6379/0'

app = Celery('task', broker=BROKER_URL, backend=CELERY_RESULT_BACKEND)

@app.task 
def prime_number(n):
    prime_count = 0
    for num1 in range(1, n+1):
        is_prime = True
        for num2 in range(2, num1):
            if num1 % num2 == 0:
                is_prime = False
        if is_prime:
            prime_count += 1
    return prime_count

Overwriting task.py


- run celery

In [18]:
import task

In [21]:
result_1 = task.prime_number.delay(10000) # delay가 작업 시작을 의미
result_2 = task.prime_number.delay(20000)

In [22]:
import time

is_done_1, is_done_2 = False, False
r_1, r_2 = 0, 0
count = 0

while (not is_done_1) or (not is_done_2):
    is_done_1 = result_1.ready() # ready로 작업상태를 확인
    is_done_2 = result_2.ready()
    time.sleep(1)
    count += 1
    
    if is_done_1:
        r_1 = result_1.get()
    if is_done_2:
        r_2 = result_2.get()
        
    print("{} sec : done1-{}, done2-{}, r_1-{}, r_2-{}".format(count, is_done_1, is_done_2, r_1, r_2))

result_1.get(), result_2.get()

1 sec : done1-False, done2-False, r_1-0, r_2-0
2 sec : done1-True, done2-False, r_1-1230, r_2-0
3 sec : done1-True, done2-False, r_1-1230, r_2-0
4 sec : done1-True, done2-False, r_1-1230, r_2-0
5 sec : done1-True, done2-False, r_1-1230, r_2-0
6 sec : done1-True, done2-False, r_1-1230, r_2-0
7 sec : done1-True, done2-False, r_1-1230, r_2-0
8 sec : done1-True, done2-False, r_1-1230, r_2-0
9 sec : done1-True, done2-False, r_1-1230, r_2-0
10 sec : done1-True, done2-False, r_1-1230, r_2-0
11 sec : done1-True, done2-False, r_1-1230, r_2-0
12 sec : done1-True, done2-True, r_1-1230, r_2-2263


(1230, 2263)